In [1]:
import tensorflow as tf
import numpy as np
import pickle

In [2]:
model_name = 'P1C1_xys'
n_input = 4
activtypes = [1,1,1,2] #1:sigmoid, 2:linear
Kvol = 1 #Kvol = 1 is the best value for Porcine data
theta = 0
a0 = np.array([ np.cos(theta), np.sin(theta), 0])
s0 = np.array([-np.sin(theta), np.cos(theta), 0])

In [3]:
with open('savednet/' + model_name + '_factors.pkl','rb') as f:
    norm_factors = pickle.load(f)
meanPsi, stdPsi = norm_factors['meanPsi'], norm_factors['stdPsi']
meanI1, stdI1   = norm_factors['meanI1'],  norm_factors['stdI1']
meanI2, stdI2   = norm_factors['meanI2'],  norm_factors['stdI2']
meanI4a, stdI4a = norm_factors['meanI4a'], norm_factors['stdI4a']
meanI4s, stdI4s = norm_factors['meanI4s'], norm_factors['stdI4s']
norm_const = np.array([meanI1, stdI1, meanI2, stdI2, meanI4a, stdI4a, meanI4s, stdI4s, meanPsi, stdPsi])
norm_const = norm_const.astype('float32')

In [4]:
model_fname   = 'savednet/'+model_name+'.json'
weights_fname = 'savednet/'+model_name+'_weights.h5'
model = tf.keras.models.model_from_json(open(model_fname).read())
model.load_weights(weights_fname)
A = model.get_weights()
weights = A[::2]
biases = A[1::2]
n_layers = len(weights) + 1 #Including the input layer
n_neuronsperlayer = [n_input] + [b.shape[0] for b in biases]

weights_flat = []
for weight in weights:
    weights_flat.append(weight.flatten())
weights_flat = np.hstack(weights_flat)

biases_flat = []
for bias in biases:
    biases_flat.append(bias.flatten())
biases_flat = np.hstack(biases_flat)

n_weights = weights_flat.shape[0]
n_biases = biases_flat.shape[0]


In [5]:
meta = np.array([n_layers, n_input, n_weights, n_biases])
printables = np.hstack([meta, n_neuronsperlayer, norm_const, weights_flat, biases_flat, activtypes, Kvol, a0, s0])
constants = printables.shape[0]
nrows = int(np.ceil(constants/8.0))

In [6]:
np.set_printoptions(linewidth=200)
with open('add_to_inp_file.txt', 'w') as f:
    f.write('*User Material, constants=' + str(constants)+'\n')
    for i in range(nrows):
        i1 = i*8
        i2 = i1 + 8
        string = np.array2string(printables[i1:i2],separator=',')
        string = string[1:-1] #To remove the brackets
        f.write(string+'\n')